In [1]:
import heapq

class MinHeap:
    def __init__(self):
        self.heap = []

    def push(self, val):
        heapq.heappush(self.heap, val)

    def pop(self):
        return heapq.heappop(self.heap)

    def peek(self):
        return self.heap[0]

    def size(self):
        return len(self.heap)

# Example usage:
min_heap = MinHeap()
min_heap.push(5)
min_heap.push(3)
min_heap.push(8)
min_heap.push(1)

print("Min element:", min_heap.peek())  # Output: 1

print("Popping elements:")
while min_heap.size() > 0:
    print(min_heap.pop())


Min element: 1
Popping elements:
1
3
5
8


In [2]:
import heapq

class PriorityQueue:
    def __init__(self):
        self.heap = []
        self.index = 0

    def push(self, priority, item):
        heapq.heappush(self.heap, (priority, self.index, item))
        self.index += 1

    def pop(self):
        return heapq.heappop(self.heap)[-1]

    def peek(self):
        return self.heap[0][-1]

    def size(self):
        return len(self.heap)

# Example usage:
priority_queue = PriorityQueue()
priority_queue.push(3, 'Task 1')
priority_queue.push(1, 'Task 2')
priority_queue.push(2, 'Task 3')

print("Next task:", priority_queue.peek())  # Output: 'Task 2'

print("Processing tasks:")
while priority_queue.size() > 0:
    print("Processing:", priority_queue.pop())


Next task: Task 2
Processing tasks:
Processing: Task 2
Processing: Task 3
Processing: Task 1


In [1]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [2]:
result = ListNode(None)

In [4]:
print(result.val, result.next)

None None


In [ ]:
# Enter your code here. Please read `IMPORTANT NOTES` section carefully before you proceed.

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# from sklearn.model_selection import cross_val_predict, cross_val_score

# Load the training data
data_train = pd.read_csv("train.csv", header=None)
# print(data_train.shape)

# Separate features (first 294 columns) and labels (last 6 columns)
X = data_train.iloc[:, :294]  # Features
y = data_train.iloc[:, 294:]  # Labels (six binary columns)

# Load the test data
data_test = pd.read_csv("test.csv", header=None)
# print(data_test.shape)

# Train individual classifiers for each of the six binary labels
predictions = []
for i in range(6):  # There are 6 scene types (binary outputs)
    # Extract the target column for the current scene type
    y_scene = y.iloc[:, i]
    
    # Split data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y_scene, test_size=0.3, random_state=42)
    
    # Use RandomForestClassifier as the model
    model = RandomForestClassifier(n_estimators=80, random_state=42)
    model.fit(X_train, y_train)

    # # Try cross-validaiton, but it is Time limit exceeded.
    # # Perform cross-validation and predict on the training set
    # y_train_pred_proba = cross_val_predict(model, X, y_scene, cv=5, method='predict_proba')[:, 1]  # Probabilities for class "1"
    # cross_val_accuracy = cross_val_score(model, X, y_scene, cv=5, scoring='accuracy')
    # print(f"Scene {i+1} cross-validation accuracy: {np.mean(cross_val_accuracy):.4f}")
    # # Train the model on the entire training data
    # model.fit(X, y_scene)
    
    # Validate the model
    y_val_pred_proba = model.predict_proba(X_val)[:, 1]  # Probabilities for class "1"
    y_val_pred = (y_val_pred_proba >= 0.55).astype(int)  # Apply 55% threshold
    acc = accuracy_score(y_val, y_val_pred)
    print(f"Accuracy for scene {i+1}: {acc:.4f}")
    
    # Predict on the test data
    y_test_pred_proba = model.predict_proba(data_test)[:, 1]  # Probabilities for class "1"
    y_test_pred = (y_test_pred_proba >= 0.55).astype(int)     # Apply 55% threshold
    # print(y_test_pred[:5])
    predictions.append(y_test_pred)
    # print(predictions[:5])
    
    
# Combine predictions for all six labels
predictions = np.array(predictions).T  # Transpose to get predictions per row
# print(predictions)

predictions_df = pd.DataFrame(predictions, columns=[f"Scene_{i+1}" for i in range(6)])
predictions_df.to_csv("prediction.csv", index=False, header=False)